# Tier 1 Grouping Analysis

## Setup Instructions

### 1. Create Virtual Environment
```bash
# Create virtual environment
python -m venv tier-1-venv

# Activate virtual environment
# On macOS/Linux:
source tier-1-venv/bin/activate
# On Windows:
# tier-1-venv\Scripts\activate
```

### 2. Install Dependencies
```bash
# Make sure virtual environment is activated
pip install -r requirements.txt
```

### 3. Create .env File
Create a `.env` file in the project root with your database credentials:

```env
DB_USER=your_username
DB_PASSWORD=your_password
DB_HOST=wg-data-rds.data.higg.org
DB_PORT=5432
DB_NAME=db_higg
```

**⚠️ Important:** Replace the placeholder values with your actual database credentials.

### 4. Start Jupyter
```bash
# Make sure virtual environment is activated
source tier-1-venv/bin/activate

# Start Jupyter
jupyter notebook
# or
jupyter lab
```

### 5. Select Kernel
- Open this notebook
- Select **"Tier 1 Analysis"** as your kernel
- Run the cells in order

## Security Notes
- **Never commit** the `.env` file to version control
- **Keep credentials secure** and don't share them
- **Use different credentials** for different environments (dev/staging/prod)

## Troubleshooting
- **Connection issues**: Verify your database credentials in `.env`
- **Kernel not found**: Make sure you've activated the virtual environment
- **Package errors**: Run `pip install -r requirements.txt` again

In [1]:
# Setup: Import libraries, load environment variables, and configure SQL file
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configuration: SQL file to execute
SQL_FILE = 'queries/facility type and pc.sql'  # Change this to run a different SQL file

# Create connection string
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

print(f"✅ Connected to: {DB_HOST}:{DB_PORT}/{DB_NAME}")
print(f"📁 SQL file to execute: {SQL_FILE}")
print("✅ Environment variables loaded successfully!")

✅ Connected to: wg-data-rds.data.higg.org:5432/db_higg
📁 SQL file to execute: queries/facility type and pc.sql
✅ Environment variables loaded successfully!


In [2]:
# Create database engine and test connection
print("🔧 Creating database engine...")

try:
    # Create engine with optimized settings
    engine = create_engine(
        connection_string,
        pool_pre_ping=True,
        pool_recycle=300,
        echo=False
    )
    
    # Test connection
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 'Connection successful!' as status, current_timestamp as time"))
        row = result.fetchone()
        
    print("✅ DATABASE CONNECTION SUCCESSFUL!")
    print(f"Status: {row[0]}")
    print(f"Connected at: {row[1]}")
    
except Exception as e:
    print("❌ DATABASE CONNECTION FAILED!")
    print(f"Error: {e}")
    print("\n🔧 Check your .env file credentials")

🔧 Creating database engine...
✅ DATABASE CONNECTION SUCCESSFUL!
Status: Connection successful!
Connected at: 2025-09-10 15:29:07.599823+00:00


In [3]:
# Execute your SQL file
print(f"📁 Loading and executing SQL file: {SQL_FILE}")

try:
    # Read SQL file using the configured variable
    with open(SQL_FILE, 'r') as file:
        sql_query = file.read()
    
    print(f"✅ SQL file loaded ({len(sql_query)} characters)")
    
    # Execute query using manual method
    with engine.connect() as connection:
        result = connection.execute(text(sql_query))
        rows = result.fetchall()
        columns = result.keys()
        
    # Create DataFrame
    df_results = pd.DataFrame(rows, columns=columns)
    
    print("✅ QUERY EXECUTED SUCCESSFULLY!")
    print(f"📊 Results: {df_results.shape[0]} rows, {df_results.shape[1]} columns")
    print(f"📋 Columns: {list(df_results.columns)}")
    
except FileNotFoundError:
    print(f"❌ SQL FILE NOT FOUND: {SQL_FILE}")
    print("🔧 Make sure the file exists in the current directory")
    print("💡 You can change the SQL_FILE variable in Cell 2 to point to a different file")
    
except Exception as e:
    print(f"❌ QUERY EXECUTION FAILED: {e}")
    print(f"\n🔍 SQL Query content from {SQL_FILE}:")
    print("=" * 50)
    print(sql_query)
    print("=" * 50)

📁 Loading and executing SQL file: queries/facility type and pc.sql
✅ SQL file loaded (1606 characters)
✅ QUERY EXECUTED SUCCESSFULLY!
📊 Results: 5438 rows, 4 columns
📋 Columns: ['assessment_id', 'finished_product_assembly_prod_vol_pcs', 'sipfacilityapparelpc', 'apparel_pc_count']


In [4]:
# Display and analyze results
if 'df_results' in locals() and not df_results.empty:
    print("📊 Dataset Overview:")
    print(f"   • Total rows: {len(df_results)}")
    print(f"   • Total columns: {len(df_results.columns)}")
    
    print("\n📋 Column Information:")
    for col in df_results.columns:
        dtype = df_results[col].dtype
        non_null = df_results[col].count()
        print(f"   • {col}: {dtype} ({non_null} non-null)")
    
    print("\n📄 First 10 rows:")
    print(df_results.head(10))
    
    print("\n💾 To save results:")
    print("df_results.to_csv('tier1_results.csv', index=False)")
    
else:
    print("❌ No results available. Run the previous cell first.")

📊 Dataset Overview:
   • Total rows: 5438
   • Total columns: 4

📋 Column Information:
   • assessment_id: object (5438 non-null)
   • finished_product_assembly_prod_vol_pcs: object (5438 non-null)
   • sipfacilityapparelpc: object (5438 non-null)
   • apparel_pc_count: int64 (5438 non-null)

📄 First 10 rows:
                                    assessment_id  \
0  femsurvey:fffff92a-914f-446f-812f-8141dbe416a6   
1  femsurvey:ffff536a-d061-4fbd-84af-bfcaf59ac297   
2  femsurvey:fff81b83-f145-404b-aeae-d0bb63b0fa1a   
3  femsurvey:fff4757f-0b72-4f89-82cb-771a86980e0f   
4  femsurvey:ffe917d7-0bf2-4469-af77-cba73a34e513   
5  femsurvey:ffe53b8f-d79c-4480-a52f-92f026b318d2   
6  femsurvey:ffba25e8-eaea-486e-9abf-6e362be0f88d   
7  femsurvey:ffa679a1-848a-465e-92f3-9d518fbe7633   
8  femsurvey:ffa0e74d-2f96-4117-b7ba-46680f7741f4   
9  femsurvey:ff8205f1-281f-4d6f-8d69-72e447f694ca   

  finished_product_assembly_prod_vol_pcs  \
0                                 400000   
1                

In [5]:
# Load PIC default product weights for apparel categories
print("📁 Loading PIC default product weights...")

try:
    # Load the CSV file (it's actually tab-separated)
    weights_df = pd.read_csv('PIC default product weights.csv', sep='\t')
    
    print("✅ PIC default product weights loaded successfully!")
    print(f"📊 Weights data: {weights_df.shape[0]} rows, {weights_df.shape[1]} columns")
    print(f"📋 Columns: {list(weights_df.columns)}")
    
    # Display the weights data (only FEM Apparel PC and Product Weight columns)
    print("\n📄 PIC Default Product Weights:")
    display_df = weights_df[['FEM Apparel PC', 'Product Weight (kg)']]
    print(display_df)
    
    # Create a dictionary mapping for easy lookup
    apparel_weights = dict(zip(weights_df['FEM Apparel PC'], weights_df['Product Weight (kg)']))
    
    print(f"\n🔗 Created weight mapping for {len(apparel_weights)} apparel categories:")
    for category, weight in apparel_weights.items():
        print(f"   • {category}: {weight} kg")
     
except FileNotFoundError:
    print("❌ PIC default product weights.csv file not found!")
    print("🔧 Make sure the file exists in the current directory")
    
except Exception as e:
    print(f"❌ Error loading weights file: {e}")


📁 Loading PIC default product weights...
✅ PIC default product weights loaded successfully!
📊 Weights data: 14 rows, 3 columns
📋 Columns: ['FEM Apparel PC', 'PIC Product', 'Product Weight (kg)']

📄 PIC Default Product Weights:
       FEM Apparel PC  Product Weight (kg)
0              Shirts             0.250000
1             Dresses             0.374213
2             Jackets             0.950000
3               Pants             0.453592
4              Skirts             0.290299
5               Socks             0.400000
6            Sweaters             0.550000
7           Swimsuits             0.100000
8          Baselayers             0.111130
9             Hosiery             0.227000
10  Leggings & Tights             0.227000
11            Jerseys             0.150000
12            T-shirt             0.150000
13          Underwear             0.138346

🔗 Created weight mapping for 14 apparel categories:
   • Shirts: 0.25 kg
   • Dresses: 0.3742134 kg
   • Jackets: 0.95 kg
   • 

In [7]:
# Calculate assumed average product weight for each assessment
print("⚖️ Calculating assumed average product weights...")

import ast  # For safely evaluating string representations of lists

def calculate_average_weight(apparel_pc_list, weights_dict):
    """
    Calculate average weight for a list of apparel product categories
    
    Args:
        apparel_pc_list: List of apparel categories (e.g., ["Shirts", "Pants"])
        weights_dict: Dictionary mapping apparel categories to weights
    
    Returns:
        Average weight in kg, or None if no valid categories found
    """
    try:
        # Handle both list and string inputs
        if isinstance(apparel_pc_list, str):
            # If it's a string, try to parse it as a list
            apparel_categories = ast.literal_eval(apparel_pc_list)
        else:
            # If it's already a list, use it directly
            apparel_categories = apparel_pc_list
        
        # Get weights for each category
        weights = []
        for category in apparel_categories:
            if category in weights_dict:
                weights.append(weights_dict[category])
            else:
                print(f"⚠️ Warning: Category '{category}' not found in weights dictionary")
        
        # Calculate average if we have valid weights
        if weights:
            return sum(weights) / len(weights)
        else:
            return None
            
    except (ValueError, SyntaxError) as e:
        print(f"⚠️ Warning: Could not parse apparel categories: {apparel_pc_list}")
        return None

# Apply the function to calculate average weights
if 'df_results' in locals() and not df_results.empty:
    print("📊 Calculating average weights for each assessment...")
    
    # Calculate average weights
    df_results['assumed_avg_product_weight_kg'] = df_results['sipfacilityapparelpc'].apply(
        lambda x: calculate_average_weight(x, apparel_weights)
    )
    
    # Convert production volume from pieces to kg using assumed average weight
    # Convert decimal values to float to avoid type mismatch
    df_results['estimated_production_volume_kg'] = (
        df_results['finished_product_assembly_prod_vol_pcs'].astype(float) * 
        df_results['assumed_avg_product_weight_kg']
    )
    
    # Add weight range category for each assessment
    def categorize_weight_range(weight):
        """Categorize weight into predefined ranges"""
        if pd.isna(weight):
            return "Unknown"
        elif 0 <= weight < 0.2:
            return "Very Light (0-0.2 kg)"
        elif 0.2 <= weight < 0.3:
            return "Light (0.2-0.3 kg)"
        elif 0.3 <= weight < 0.4:
            return "Medium-Light (0.3-0.4 kg)"
        elif 0.4 <= weight < 0.5:
            return "Medium (0.4-0.5 kg)"
        elif 0.5 <= weight < 0.6:
            return "Medium-Heavy (0.5-0.6 kg)"
        elif weight >= 0.6:
            return "Heavy (0.6+ kg)"
        else:
            return "Unknown"
    
    df_results['weight_range_category'] = df_results['assumed_avg_product_weight_kg'].apply(categorize_weight_range)
    
    # Display summary statistics
    valid_weights = df_results['assumed_avg_product_weight_kg'].dropna()
    
    print(f"✅ Average weights calculated!")
    print(f"📊 Summary statistics:")
    print(f"   • Total assessments: {len(df_results)}")
    print(f"   • Assessments with valid weights: {len(valid_weights)}")
    print(f"   • Assessments with missing weights: {len(df_results) - len(valid_weights)}")
    
    if len(valid_weights) > 0:
        print(f"   • Average weight across all assessments: {valid_weights.mean():.4f} kg")
        print(f"   • Min weight: {valid_weights.min():.4f} kg")
        print(f"   • Max weight: {valid_weights.max():.4f} kg")
        print(f"   • Median weight: {valid_weights.median():.4f} kg")
    
    # Production volume statistics
    valid_production_kg = df_results['estimated_production_volume_kg'].dropna()
    if len(valid_production_kg) > 0:
        print(f"\n📦 Production Volume Statistics (in kg):")
        print(f"   • Total estimated production volume: {valid_production_kg.sum():,.0f} kg")
        print(f"   • Average production per assessment: {valid_production_kg.mean():,.0f} kg")
        print(f"   • Median production per assessment: {valid_production_kg.median():,.0f} kg")
        print(f"   • Min production: {valid_production_kg.min():,.0f} kg")
        print(f"   • Max production: {valid_production_kg.max():,.0f} kg")
    
    # Show some examples
    print(f"\n📄 Sample results (first 10 rows):")
    sample_cols = ['assessment_id', 'sipfacilityapparelpc', 'apparel_pc_count', 
                   'finished_product_assembly_prod_vol_pcs', 'assumed_avg_product_weight_kg', 
                   'weight_range_category', 'estimated_production_volume_kg']
    print(df_results[sample_cols].head(10))
    
    # Show weight distribution using the new category column
    print(f"\n📊 Weight Range Distribution:")
    weight_category_counts = df_results['weight_range_category'].value_counts()
    total_assessments = len(df_results)
    
    for category in ["Very Light (0-0.2 kg)", "Light (0.2-0.3 kg)", "Medium-Light (0.3-0.4 kg)", 
                     "Medium (0.4-0.5 kg)", "Medium-Heavy (0.5-0.6 kg)", "Heavy (0.6+ kg)", "Unknown"]:
        count = weight_category_counts.get(category, 0)
        pct = (count / total_assessments) * 100
        print(f"   • {category}: {count} assessments ({pct:.1f}%)")
    
    # Export results to CSV in Documents folder
    import os
    from datetime import datetime
    
    # Create filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"tier1_analysis_results_{timestamp}.csv"
    
    # Get Documents folder path
    documents_path = os.path.expanduser("~/Documents")
    filepath = os.path.join(documents_path, filename)
    
    try:
        # Export the DataFrame with all columns
        df_results.to_csv(filepath, index=False)
        print(f"\n💾 Results exported successfully!")
        print(f"📁 File saved to: {filepath}")
        print(f"📊 Exported {len(df_results)} rows with {len(df_results.columns)} columns")
        print(f"📋 Columns exported: {list(df_results.columns)}")
        
    except Exception as e:
        print(f"\n❌ Export failed: {e}")
        print("💡 Trying alternative export to current directory...")
        try:
            # Fallback to current directory
            fallback_path = f"tier1_analysis_results_{timestamp}.csv"
            df_results.to_csv(fallback_path, index=False)
            print(f"✅ Fallback export successful: {fallback_path}")
        except Exception as e2:
            print(f"❌ Fallback export also failed: {e2}")
    
else:
    print("❌ No results data available. Run the previous cells first to load the data.")


⚖️ Calculating assumed average product weights...
📊 Calculating average weights for each assessment...
✅ Average weights calculated!
📊 Summary statistics:
   • Total assessments: 5438
   • Assessments with valid weights: 5438
   • Assessments with missing weights: 0
   • Average weight across all assessments: 0.3776 kg
   • Min weight: 0.1000 kg
   • Max weight: 0.9500 kg
   • Median weight: 0.3740 kg

📦 Production Volume Statistics (in kg):
   • Total estimated production volume: 162,656,150,094 kg
   • Average production per assessment: 29,911,024 kg
   • Median production per assessment: 678,729 kg
   • Min production: 0 kg
   • Max production: 120,932,688,000 kg

📄 Sample results (first 10 rows):
                                    assessment_id  \
0  femsurvey:fffff92a-914f-446f-812f-8141dbe416a6   
1  femsurvey:ffff536a-d061-4fbd-84af-bfcaf59ac297   
2  femsurvey:fff81b83-f145-404b-aeae-d0bb63b0fa1a   
3  femsurvey:fff4757f-0b72-4f89-82cb-771a86980e0f   
4  femsurvey:ffe917d7-0bf